In [9]:
import re 
import nltk 
import numpy as np 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer ## stemming 
from nltk.stem import WordNetLemmatizer  ## lemmatization
import joblib,os 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences 
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
from sklearn.model_selection import train_test_split

print('libraries imported 😎')


libraries imported 😎


In [10]:
## Reading data from the text files 

train_data = open("./Data/train.txt").readlines()
val_data = open("./Data/val.txt").readlines()
test_data = open("./Data/test.txt").readlines()


In [11]:
print(len(train_data))
print(len(val_data))
print(len(test_data))


16000
2000
2000


In [12]:
complet_data = train_data + test_data + val_data 
print(len(complet_data))

20000


In [13]:
x=[]
y=[]
for temp in complet_data:
    complete = temp.split(';')
    if len(complete)==2:
        x.append(complete[0])
        y.append(complete[1].strip())

In [14]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
labels 

['sadness', 'anger', 'love', 'surprise', 'fear', 'joy']

In [15]:
## Text cleaning 
# 1. lower case convert all message 
# 2. a-z0-9   other characters 
# 3. 

In [16]:
text cleaning function applying on each and every raw message 
def text_cleaning(sentences,stemming):
    cleaned_data = []
    for sentece in sentences: 
        message = sentece.lower()
        message = re.sub('[^a-z0-9 ]',"",message)
        ls_of_words = nltk.word_tokenize(message)
        ls_of_word_without_stopwords = [word for word in ls_of_words if word not in stopwords.words('english')]
        stemmed_words = [stemming.stem(word) for word in ls_of_word_without_stopwords ]
        message = " ".join(stemmed_words)
        cleaned_data.append(message)
    return cleaned_data

stem = PorterStemmer()
cleaned_data = text_cleaning(x,stemming=stem)


In [17]:
os.makedirs('models',exist_ok=True)
joblib.dump(cleaned_data,"./models/cleaned_data.lb")
# cleaned_data = joblib.load("./models/cleaned_data.lb")
print("successfully saved your cleaned data!")

successfully saved your cleaned data!


In [18]:
## trying  on a sinlgle message 
ls_of_stopwords = stopwords.words('english')
print(x[1])
single_message = x[1].lower()
single_message = re.sub('[^a-z0-9 ]',"",single_message)
ls_of_words = nltk.word_tokenize(single_message)  # single_message.split()
ls_of_word_without_stopwords = []
for word in ls_of_words: 
    if word not in ls_of_stopwords:
        ls_of_word_without_stopwords.append(word)
print(ls_of_word_without_stopwords)
stem = PorterStemmer()
stemmed_words = []
for word in ls_of_word_without_stopwords: 
    stemmed_words.append(stem.stem(word))
print(stemmed_words)
" ".join(stemmed_words)

i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
['go', 'feeling', 'hopeless', 'damned', 'hopeful', 'around', 'someone', 'cares', 'awake']
['go', 'feel', 'hopeless', 'damn', 'hope', 'around', 'someon', 'care', 'awak']


'go feel hopeless damn hope around someon care awak'

In [19]:
## Tokenaization 
tokenizer = Tokenizer(oov_token="<nothing>")
tokenizer.fit_on_texts(cleaned_data)
joblib.dump(tokenizer,'./models/tokenizer.lb')
print("successfully saved your tokenizer at this location : './models/tokenizer.lb'")

successfully saved your tokenizer at this location : './models/tokenizer.lb'


In [20]:
# tokenizer.word_index

In [40]:
# tokenizer.word_counts  

In [22]:
tokenized_data = tokenizer.texts_to_sequences(cleaned_data)

In [23]:
len(tokenized_data)

20000

In [24]:
cleaned_data[0]

'didnt feel humili'

In [25]:
tokenized_data[0]

[61, 2, 522]

In [26]:
# tokenizer.word_index

In [27]:
len_of_message = []
for i in range(len(tokenized_data)):
    len_of_message.append(len(tokenized_data[i]))
print("Your maximum length is : ",max(len_of_message))
# maximum length  

Your maximum length is :  35


In [28]:

print("Your maximum length is : ",max(list(map(len,tokenized_data))))

Your maximum length is :  35


In [29]:
sequences = pad_sequences(tokenized_data,maxlen=35,padding='post')


In [30]:
sequences

array([[  61,    2,  522, ...,    0,    0,    0],
       [  10,    2,  419, ...,    0,    0,    0],
       [   4, 1230,  431, ...,    0,    0,    0],
       ...,
       [   2,  194,  157, ...,    0,    0,    0],
       [ 328,    2,  175, ...,    0,    0,    0],
       [   2,    3,  916, ...,    0,    0,    0]])

In [31]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
label_dict = {label:i for i , label in enumerate(labels)}
label_dict


{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}

In [32]:
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
# automation 

In [33]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)

In [34]:
Y

array([4, 4, 0, ..., 2, 2, 2], dtype=int64)

In [35]:
label_encoder.classes_

array(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'], dtype='<U8')

In [36]:
joblib.dump(label_encoder,'./models/label_encoder.lb')

['./models/label_encoder.lb']

In [37]:
sequences    # <====  X DATA  cleaned data 
Y            # <====  Y DATA label 


array([4, 4, 0, ..., 2, 2, 2], dtype=int64)

In [38]:
## train test splitting 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(sequences,Y,test_size=0.15)

In [43]:
# Define the model
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM , Dense

model = Sequential([
    LSTM(units=128, input_shape=(35, 1), return_sequences=True),
    LSTM(units=64),
    Dense(units=6, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\Ranjit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 115s 169ms/step - accuracy: 0.3275 - loss: 1.5959 - val_accuracy: 0.3217 - val_loss: 1.5773
Epoch 2/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 163s 206ms/step - accuracy: 0.3353 - loss: 1.5800 - val_accuracy: 0.3410 - val_loss: 1.5700
Epoch 3/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 113s 150ms/step - accuracy: 0.3357 - loss: 1.5752 - val_accuracy: 0.3460 - val_loss: 1.5678
Epoch 4/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 87s 163ms/step - accuracy: 0.3257 - loss: 1.5759 - val_accuracy: 0.3410 - val_loss: 1.5700
Epoch 5/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 114s 213ms/step - accuracy: 0.3422 - loss: 1.5699 - val_accuracy: 0.3503 - val_loss: 1.5681


In [ ]:
# from sklearn.ensemble import RandomForestCLassifier 
# obj = RandomForestCLassifier()
# obj.fit(x_train,y_train)

In [46]:
# prediction 
prediction = model.predict(x_test)

94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step


In [50]:
import numpy as np 
np.argmax(prediction,axis=0)

array([ 865, 1403, 2457, 2029, 1403, 2306], dtype=int64)

In [57]:
prediction = np.argmax(prediction,axis=1)

In [58]:
prediction 

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [59]:
y_test

array([4, 1, 2, ..., 4, 2, 2], dtype=int64)

In [62]:
import pandas as pd 
df = pd.DataFrame({'actual':y_test,'prediction':prediction})

In [69]:
label_dictionary = {value:key for key , value in label_dict.items()}
label_dictionary

{0: 'sadness', 1: 'anger', 2: 'love', 3: 'surprise', 4: 'fear', 5: 'joy'}

In [71]:
df['actual'] = df['actual'].map(label_dictionary)
df['prediction'] = df['prediction'].map(label_dictionary)


In [72]:
df.head(40)

,actual,prediction
0,fear,love
1,anger,love
2,love,love
3,love,love
4,fear,love
5,fear,love
6,sadness,fear
7,joy,love
8,fear,love
9,love,love


In [ ]:
### using RNN 
# sequential SimpleRNN model defining
from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(SimpleRNN(32,input_shape=(35,1),return_sequences=False))
model.add(Dense(6,activation='softmax'))



## compiling the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
# return_sequence = False  --> means if you don,t want to stack again this model then specify False

In [73]:
## model save
import os
os.makedirs('models',exist_ok=True)  
model.save('./models/lstm_model.h5')

In [74]:
## to load the tensorflow models 
from tensorflow.keras.models import  load_model 
loaded_model = load_model('./models/lstm_model.h5')

In [76]:
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape           ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ lstm (LSTM)                    │ (None, 35, 128)        │       66,560 │
├────────────────────────────────┼────────────────────────┼──────────────┤
│ lstm_1 (LSTM)                  │ (None, 64)             │       49,408 │
├────────────────────────────────┼────────────────────────┼──────────────┤
│ dense (Dense)                  │ (None, 6)              │          390 │
└────────────────────────────────┴────────────────────────┴──────────────┘

 Total params: 116,360 (454.54 KB)

 Trainable params: 116,358 (454.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)